In [49]:
from dataclasses import dataclass, field
import pandas as pd
import numpy as np
import json
import os 
import sys
sys.path.append("/Users/atticussoane/Desktop/atticus_tools/")
from data_science import PrepTools, ModelSelectionTools
from bankroll_calc import BankrollCalculator
from recursive_selection import FeatureSelector
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# IDEAS TO TRY TODAY:

* REMOVE GAME 1s and / or large outliers - DIDNT' WORK, 60% accuracy (as high as I can get) does not lead to profitable strategy.
* COMBINE items from each feature set 
* NO GAMBLING features at all - try to predict solely based on statistics - Get creative here, maybe try to create your own rating system.
* Generate probability distribution from Neural Network with custom loss function 
* Time Permitting- Let's look at the OVER / UNDER

In [2]:
adv = pd.read_csv("./adv_metrics/current.csv.gz", compression = "gzip")

pd.set_option("max.columns", 200) 

adv.head()

,date,home_close_TJ,home_close_ou,home_closing,home_open_TJ,home_open_ou,home_opening,home_runline,home_runline_juice,is_doubleheader,road_close_TJ,road_close_ou,road_closing,road_open_TJ,road_open_ou,road_opening,road_runline,road_runline_juice,is_tripleheader,season,home_team,road_team,home_starter,road_starter,home_wOBA,home_wRAA,home_wRC,home_OPS,home_relief_wOBA,home_relief_wRAA,home_relief_wRC,home_relief_FIP,home_relief_WHIP,home_relief_ERA,home_relief_K_BB,home_relief_K_9,home_starter_career_wOBA,home_starter_career_wRAA,home_starter_career_wRC,home_starter_career_FIP,home_starter_career_WHIP,home_starter_career_ERA,home_starter_careerK/BB,home_starter_careerK/9,home_starter_career_AVGIP,home_starter_season_wOBA,home_starter_season_wRAA,home_starter_season_wRC,home_starter_season_FIP,home_starter_season_WHIP,home_starter_season_ERA,home_starter_seasonK/BB,home_starter_seasonK/9,home_starter_seasonAVG_IP,road_wOBA,road_wRAA,road_wRC,road_OPS,road_relief_wOBA,road_relief_wRAA,road_relief_wRC,road_relief_FIP,road_relief_WHIP,road_relief_ERA,road_relief_K_BB,road_relief_K_9,road_starter_career_wOBA,road_starter_career_wRAA,road_starter_career_wRC,road_starter_career_FIP,road_starter_career_WHIP,road_starter_career_ERA,road_starter_careerK/BB,road_starter_careerK/9,road_starter_career_AVGIP,road_starter_season_wOBA,road_starter_season_wRAA,road_starter_season_wRC,road_starter_season_FIP,road_starter_season_WHIP,road_starter_season_ERA,road_starter_seasonK/BB,road_starter_seasonK/9,road_starter_seasonAVG_IP,home_payout,road_payout,elo1_pre,elo2_pre,rating1_pre,rating2_pre,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,score1,score2,home_win,home_favorite,line_movement,money_scalar,batting_park_factor,home_streak,home_H_streak,home_record,run_differential_hm,avg_margin_hm,distance_traveled,road_streak,road_R_streak,road_record,run_differential_rd,avg_margin_rd
0,2010-04-04,-115,9.0,-117.0,-104,9.0,-114.0,NaN,NaN,0,-105,9.0,107.0,-116,9.0,104.0,NaN,NaN,0.0,2010.0,BOS,NYY,beckj002,sabac001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.854701,2.070000,1537.262,1568.875,1536.601,1562.673,57.031,61.828,27.349300,46.7039,0.471714,9.0,7.0,1,1,1,-3.0,106.0,0.0,0.0,0.0,0.0,0.0,289.0,0.0,0.0,0.0,0.0,0.0
1,2010-04-05,105,7.5,175.0,-120,7.5,175.0,NaN,NaN,0,-125,7.5,-200.0,100,7.5,-200.0,NaN,NaN,0.0,2010.0,WSN,PHI,lannj001,hallr001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.750000,1.500000,1460.859,1541.211,1460.662,1537.875,49.834,62.166,8.394200,49.4957,0.366599,1.0,11.0,0,0,0,0.0,99.0,0.0,0.0,0.0,0.0,0.0,195.0,0.0,0.0,0.0,0.0,0.0
2,2010-04-05,-105,7.0,137.0,-105,7.0,110.0,NaN,NaN,0,-115,7.0,-152.0,-115,7.0,-125.0,NaN,NaN,0.0,2010.0,OAK,SEA,sheeb001,hernf002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.370000,1.657895,1509.222,1501.778,1513.910,1500.945,56.701,60.524,7.593298,46.4172,0.501246,3.0,5.0,0,0,-1,27.0,99.0,0.0,0.0,0.0,0.0,0.0,1095.0,0.0,0.0,0.0,0.0,0.0
3,2010-04-05,110,8.5,-128.0,100,8.5,-135.0,NaN,NaN,0,-130,8.5,113.0,-120,8.5,120.0,NaN,NaN,0.0,2010.0,ANA,MIN,weavj003,bakes002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.781250,2.130000,1544.703,1521.978,1543.114,1520.771,54.034,55.637,16.337200,26.8276,0.550540,6.0,3.0,1,1,-1,7.0,96

In [3]:
adv.describe()

,home_close_TJ,home_close_ou,home_closing,home_open_TJ,home_open_ou,home_opening,home_runline,home_runline_juice,is_doubleheader,road_close_TJ,road_close_ou,road_closing,road_open_TJ,road_open_ou,road_opening,road_runline,road_runline_juice,is_tripleheader,season,home_wOBA,home_wRAA,home_wRC,home_OPS,home_relief_wOBA,home_relief_wRAA,home_relief_wRC,home_relief_FIP,home_relief_WHIP,home_relief_ERA,home_relief_K_BB,home_relief_K_9,home_starter_career_wOBA,home_starter_career_wRAA,home_starter_career_wRC,home_starter_career_FIP,home_starter_career_WHIP,home_starter_career_ERA,home_starter_careerK/BB,home_starter_careerK/9,home_starter_career_AVGIP,home_starter_season_wOBA,home_starter_season_wRAA,home_starter_season_wRC,home_starter_season_FIP,home_starter_season_WHIP,home_starter_season_ERA,home_starter_seasonK/BB,home_starter_seasonK/9,home_starter_seasonAVG_IP,road_wOBA,road_wRAA,road_wRC,road_OPS,road_relief_wOBA,road_relief_wRAA,road_relief_wRC,road_relief_FIP,road_relief_WHIP,road_relief_ERA,road_relief_K_BB,road_relief_K_9,road_starter_career_wOBA,road_starter_career_wRAA,road_starter_career_wRC,road_starter_career_FIP,road_starter_career_WHIP,road_starter_career_ERA,road_starter_careerK/BB,road_starter_careerK/9,road_starter_career_AVGIP,road_starter_season_wOBA,road_starter_season_wRAA,road_starter_season_wRC,road_starter_season_FIP,road_starter_season_WHIP,road_starter_season_ERA,road_starter_seasonK/BB,road_starter_seasonK/9,road_starter_seasonAVG_IP,home_payout,road_payout,elo1_pre,elo2_pre,rating1_pre,rating2_pre,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,score1,score2,home_win,home_favorite,line_movement,money_scalar,batting_park_factor,home_streak,home_H_streak,home_record,run_differential_hm,avg_margin_hm,distance_traveled,road_streak,road_R_streak,road_record,run_differential_rd,avg_margin_rd
count,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,14346.000000,14346.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,14346.000000,14346.000000,23939.0,23939.00000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,2.393900e+04,2.393900e+04,2.393900e+04,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000,23939.000000
mean,-71.006642,8.345106,-69.733531,-81.804587,8.360896,-68.641422,-0.462254,30.419350,0.004261,-67.861815,8.345106,37.805339,-70.272401,8.360896,37.731693,0.462254,-70.997560,0.0,2014.49576,0.312362,-1.969094,94.964994,0.718946,0.305602,-5.968401,90.337332,3.918384,1.296942,3.808963,2.741298,8.369287,0.295920,-1.103617,90.414796,inf,inf,inf,2.901102,6.996301,5.433249,0.294497,-1.140735,89.881280,3.897796,1.228980,3.886043,2.906347,6.969659,5.414108,0.312175,-1.627884,94.835366,0.718533,0.306096,-5.943094,90.675806,3.926917,1.300081,3.823707,2.744333,8.359416,0.294232,-1.266199,89.298422,3.878704,1.225912,3.857053,2.932210,7.032893,5.450130,0.292975,-1.294415,88.799522,3.862127,1.220633,3.836821,2.939160,7.011520,5.434138,1.8

# STRATEGY: REMOVING GAME ONES

* PART A- REMOVE GAMBLING DATA
* PART B- PUT GAMBLING DATA BACK (LINE MOVEMENT, LINE SCALAR)

In [4]:
game1_indices = adv[(adv.home_wOBA == 0.) | (adv.road_wOBA == 0.)].index

print(len(game1_indices))

150


In [5]:
g1 = adv.drop(game1_indices, axis = 0).reset_index(drop = True)

In [6]:
drop_cols = ["home_close_TJ", "home_close_ou", "home_closing", "home_open_TJ", "home_open_ou", "home_runline",
            "home_runline_juice", "is_doubleheader", "road_close_TJ", "road_close_ou", "road_closing",
            "road_open_TJ", "road_open_ou", "road_runline", "road_runline_juice", "is_tripleheader", "home_team",
            "road_team", "home_starter", "road_starter", "road_payout", "elo1_pre", "elo2_pre", "rating_prob1",
            "score1", "score2", "home_favorite", "line_movement", "money_scalar"]

with open("./adv_metrics/adv_metrics_drop_cols.json", "w+") as f:
    json.dump(drop_cols, f)

In [7]:
g1 = g1.drop(4377, axis = 0).reset_index(drop = True)

g1 = g1.drop(columns = drop_cols)

In [8]:
train = g1[g1.season < 2019].drop(columns = ["season", "date"])

test = g1[g1.season == 2019].drop(columns = ["season", "date"]).reset_index(drop = True)

test_dates = adv[adv.season == 2019].date.reset_index(drop = True)

test_hc = adv[adv.season == 2019].home_closing.reset_index(drop = True)

test_rc = adv[adv.season == 2019].road_closing.reset_index(drop = True)

In [9]:
ytrain, train = train.home_win, train.drop("home_win", axis = 1)

ytest, test = test.home_win, test.drop("home_win", axis = 1)

In [10]:
train.shape, ytrain.shape, test.shape, ytest.shape

((21419, 81), (21419,), (2369, 81), (2369,))

## FEATURE SELECTION

In [11]:
train, ss = PrepTools(train, ytrain).scale_features(save_scaler = True)

test = PrepTools(test, ytest).scale_features(load_scaler = ss)

In [13]:
l1_params = {
    
    "penalty" : "l1",
    
    "solver" : "liblinear",
    
    "C" : 1.0,
    
    "fit_intercept" : False
}

selector_params = {
    
    "X" : train,
    
    "y" : ytrain,
    
    "algorithm" : LogisticRegression(),
    
    "params" : l1_params
}

selector = FeatureSelector(**selector_params)

In [14]:
selector.recursive_selection()

'Cannot reduce feature frame anymore. Reduce drop size if desired'

In [16]:
selector.best_subset

Index(['road_opening'], dtype='object')

In [17]:
l2_param = {
    
    "penalty" : "l2",
    
    "solver" : "lbfgs",
    
    "C" : 1.0,
    
    "fit_intercept" : False,
    
    "max_iter" : 1000
}

selector_params = {
    
    "X" : train,
    
    "y" : ytrain,
    
    "algorithm" : LogisticRegression(),
    
    "params" : l2_param
}

selector = FeatureSelector(**selector_params)

selector.recursive_selection()

'Cannot reduce feature frame anymore. Reduce drop size if desired'

In [19]:
selector.best_subset

Index(['road_opening'], dtype='object')

In [23]:
lgb_params = {
    
    "boosting_type" : "gbdt",
    
    "max_depth" : 3,
    
    "num_leaves" : 36,
    
    "n_estimators" : 100,
    
    "objective" : "binary",
    
    "reg_alpha" : 0.5,
    
    "reg_lambda" : 0.5
}

selector_params = {
    
    "X" : train,
    
    "y" : ytrain,
    
    "algorithm" : LGBMClassifier(),
    
    "params" : lgb_params
}

selector = FeatureSelector(**selector_params)

selector.recursive_selection()

'Cannot reduce feature frame anymore. Reduce drop size if desired'

In [25]:
selector.best_subset

Index(['home_relief_wRAA', 'road_relief_K_BB', 'home_payout', 'pitcher2_rgs'], dtype='object')

In [26]:
xgb_params = {
    
    "max_depth" : 3,
    
    "reg_alpha" : 0.5,
    
    "reg_lambda" : 0.5, 
    
    "colsample_by_tree" : 0.6
}

selector_params = {
    
    "X" : train,
    
    "y" : ytrain,
    
    "algorithm" : XGBClassifier(),
    
    "params" : xgb_params
}

selector = FeatureSelector(**selector_params)

selector.recursive_selection()

'Cannot reduce feature frame anymore. Reduce drop size if desired'

In [30]:
selector.best_subset

Index(['home_opening', 'road_opening', 'home_relief_wOBA', 'home_relief_wRAA',
       'home_relief_wRC', 'home_relief_ERA', 'home_starter_career_wOBA',
       'home_starter_career_wRAA', 'home_starter_career_wRC',
       'home_starter_season_wRAA', 'home_starter_season_wRC',
       'home_starter_seasonAVG_IP', 'road_wRAA', 'road_wRC', 'road_OPS',
       'road_relief_wOBA', 'road_relief_wRAA', 'road_relief_wRC',
       'road_relief_FIP', 'road_relief_WHIP', 'road_relief_ERA',
       'road_relief_K_9', 'road_starter_career_wRAA',
       'road_starter_career_wRC', 'road_starter_season_wOBA',
       'road_starter_season_wRAA', 'road_starter_season_wRC',
       'road_starter_season_FIP', 'road_starter_season_ERA',
       'road_starter_seasonK/9', 'home_payout', 'rating1_pre', 'rating2_pre',
       'pitcher1_rgs', 'pitcher2_rgs', 'run_differential_rd'],
      dtype='object')

#### LETS TRY SOME MODEL BUILDING BEFORE WE SLING THESE THROUGH THE OLD NEURAL NETWORK

In [31]:
log_ = ["road_opening"]

lgb_ = ['home_relief_wRAA', 'road_relief_K_BB', 'home_payout', 'pitcher2_rgs']

xgb_ = ['home_opening', 'road_opening', 'home_relief_wOBA', 'home_relief_wRAA',
       'home_relief_wRC', 'home_relief_ERA', 'home_starter_career_wOBA',
       'home_starter_career_wRAA', 'home_starter_career_wRC',
       'home_starter_season_wRAA', 'home_starter_season_wRC',
       'home_starter_seasonAVG_IP', 'road_wRAA', 'road_wRC', 'road_OPS',
       'road_relief_wOBA', 'road_relief_wRAA', 'road_relief_wRC',
       'road_relief_FIP', 'road_relief_WHIP', 'road_relief_ERA',
       'road_relief_K_9', 'road_starter_career_wRAA',
       'road_starter_career_wRC', 'road_starter_season_wOBA',
       'road_starter_season_wRAA', 'road_starter_season_wRC',
       'road_starter_season_FIP', 'road_starter_season_ERA',
       'road_starter_seasonK/9', 'home_payout', 'rating1_pre', 'rating2_pre',
       'pitcher1_rgs', 'pitcher2_rgs', 'run_differential_rd']

In [37]:
l1_params = {
    
    "penalty" : "l2",
    
    "solver" : "lbfgs",
    
    "C" : 1.0,
    
    "fit_intercept" : False,
    
    "max_iter" : 1000
}

feature_sets = [log_, lgb_, xgb_]

for i, feature in enumerate(feature_sets):
    
    lr = LogisticRegression().set_params(**l1_params)
    
    lr.fit(train[feature], ytrain)
    
    ypreds = lr.predict(test[feature])
    
    print("Accuracy of feature set {} is: {}:".format(i, round(accuracy_score(ytest, ypreds), 4)))

Accuracy of feature set 0 is: 0.5965:
Accuracy of feature set 1 is: 0.6019:
Accuracy of feature set 2 is: 0.5973:


In [38]:
lgb_params = {
    
    "boosting_type" : "gbdt",
    
    "max_depth" : 3,
    
    "num_leaves" : 36,
    
    "n_estimators" : 100,
    
    "objective" : "binary",
    
    "reg_alpha" : 0.5,
    
    "reg_lambda" : 0.5
}

for i, feature in enumerate(feature_sets):
    
    lgb = LGBMClassifier().set_params(**lgb_params)
    
    lgb.fit(train[feature], ytrain)
    
    ypreds = lgb.predict(test[feature])
    
    print("Accuracy of feature set {} is: {}:".format(i, round(accuracy_score(ytest, ypreds), 4)))

Accuracy of feature set 0 is: 0.596:
Accuracy of feature set 1 is: 0.5981:
Accuracy of feature set 2 is: 0.591:


In [46]:
xgb_params = {
    
    "max_depth" : 1,
    
    "reg_alpha" : 1.,
    
    "reg_lambda" : 1., 
    
    "colsample_by_tree" : 0.6
}

for i, feature in enumerate(feature_sets):
    
    xgb = XGBClassifier().set_params(**xgb_params)
    
    xgb.fit(train[feature], ytrain)
    
    ypreds = xgb.predict(test[feature])
    
    print("Accuracy of feature set {} is: {}:".format(i, round(accuracy_score(ytest, ypreds), 4)))

Accuracy of feature set 0 is: 0.5969:
Accuracy of feature set 1 is: 0.6019:
Accuracy of feature set 2 is: 0.6045:


Again, $60.45\%$ accuracy is what we expected to see. And as of yet, we have not been able to arbitrage a winning betting strategy from this but let's look into it one more time before we move on to the ANN.

In [47]:
win_probs = xgb.predict_proba(test[xgb_])

win_probs = [i[1] for i in win_probs]

In [80]:
@dataclass
class LinePredictor():
    predict_proba: list
    labels: pd.core.frame.Series
    home_payouts: pd.core.frame.Series
    road_payouts: pd.core.frame.Series
    dates: list or pd.core.frame.Series
    frame: pd.core.frame.DataFrame = None
    plays: list = field(default_factory = list)
    arbitrage_tol: float = 0.25
        
    def arbitrage(self):
        
        self.assemble_frame()
        
        for k in range(len(self.frame)):
            
            if self.frame.iloc[k]["model_preds"] < 2:
                
                if self.frame.iloc[k]["model_preds"] < self.frame.iloc[k]["home_payout"]:
                    
                    self.plays.append(1)
                    
                elif self.frame.iloc[k]["model_preds"] - self.frame.iloc[k]["home_payout"] > self.arbitrage_tol:
                    
                    self.plays.append(0)
                    
                else:
                    
                    self.plays.append(1)
                    
            else:
                
                if self.frame.iloc[k]["model_preds"] > self.frame.iloc[k]["home_payout"]:
                    
                    self.plays.append(0)
                    
                elif self.frame.iloc[k]["home_payout"] - self.frame.iloc[k]["model_preds"] > self.arbitrage_tol:
                    
                    self.plays.append(1)
                    
                else:
                    
                    self.plays.append(0)
                    
        plays = pd.Series(self.plays).rename("model_plays")
        
        self.frame = pd.concat([self.frame, plays], axis = 1)
                    
    def assemble_frame(self):
        
        payouts = self.generate_payouts(self.predict_proba)
        
        payouts = pd.Series(payouts).rename("model_preds")
        
        assert len(payouts) == len(self.labels) == len(self.home_payouts) ==\
        len(self.road_payouts) == len(self.dates)
        
        self.frame = pd.concat([payouts, self.labels, self.home_payouts,
                               self.road_payouts, self.dates], axis = 1)
        
    @staticmethod
    def generate_payouts(predict_proba):
        
        payouts = []
        
        for k in predict_proba:
            
            payouts.append(1 / k)
            
        return(payouts)

In [71]:
hp = g1[g1.season == 2019].home_payout.reset_index(drop = True)

rp = adv.drop(game1_indices, axis = 0)

rp = rp[rp.season == 2019].road_payout.reset_index(drop = True)

td = adv.drop(game1_indices, axis = 0)

td = td[td.season == 2019].date.reset_index(drop = True)

In [81]:
lp = LinePredictor(win_probs, ytest, hp, rp, td)

lp.arbitrage()

In [82]:
lp.frame

,model_preds,home_win,home_payout,road_payout,date,model_plays
0,1.883963,1,1.826446,2.110000,2019-03-29,1
1,2.192153,0,1.990099,1.917431,2019-03-29,0
2,2.332972,0,2.290000,1.719424,2019-03-29,0
3,2.458579,0,2.420000,1.657895,2019-03-29,0
4,2.031835,0,1.980392,1.925926,2019-03-29,0
...,...,...,...,...,...,...
2364,3.006892,1,2.700000,1.540541,2019-09-29,0
2365,2.241251,0,2.050000,1.869565,2019-09-29,0
2366,1.586629,0,1.606061,2.550000,2019-09-29,1
2367,2.614252,0,2.550000,1.606061,2019-09-29,0


In [86]:
profit = 0

for k in range(len(lp.frame)):
    
    if lp.frame.iloc[k]["home_win"] == lp.frame.iloc[k]["model_plays"] == 1:
        
        profit += lp.frame.iloc[k]["home_payout"] - 1
        
    elif lp.frame.iloc[k]["home_win"] == lp.frame.iloc[k]["model_plays"] == 0:
        
        profit += lp.frame.iloc[k]["road_payout"] - 1
        
    else:
        
        profit -= 1

In [88]:
profit * 100

-917.0118018295391